In [1]:
!pip install -q pandas==1.3.5

In [2]:
import doctest
import itertools
import string

import pandas as pd

from collections import defaultdict, Counter
from typing import Callable, Dict, List

# Week 1 - The Inverted Index & Boolean Retrieval

Welcome to the first assignment of the search engines course 👋

The assignment has two mandatory and one optional part. Part I contains questions to answer freely by text. In part II, we build an inverted index and code a small boolean search engine for Netflix content. And lastly, part III is an optional bonus task to extend the search engine to search specifically for genres, actors, etc. You only need to submit parts I and II to pass.

For any questions, problems, or feedback please contact your TA. Good luck with the assignment!


### Resources

📚 [Croft, Metzler, and Strohman - Chapter 1](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf#page=28)

📚 [Croft, Metzler, and Strohman - Chapter 7.1](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf#page=257)

📚 [Manning, Raghavan, Schütze - Chapter 1](https://nlp.stanford.edu/IR-book/pdf/01bool.pdf)

# Part I - Intro to IR

## 1.1 - Information Needs

📝 Explain the difference between a user's search query and their information need. Give an example of a query that fits three potential information needs.

A query is an (attempted) expression of a user's underlying information need. The query `bat` might, among others refer to information needs about:

- The animal
- The sports equipment
- A crypto currency
- British American Tobacco
- A biker gang from Luxembourg

## 1.2 - Vocabulary Mismatch

The vocabulary mismatch problem occurs when users refer to the same concept with different words. A user might search for a new "notebook" and expect to find websites that only mention the term "laptop".

📝 Think about potential causes of the vocabulary mismatch problem. List three potential causes and explain each with an example.

- Synonyms (aircraft, plane)
- Spelling errors (mississippi, misissippi)
- Different languages (hagelslag, sprinkles)
- Inflections (go, went, gone, etc.)
- New names (Burma, Myanmar)

## 1.3 - Boolean Retrieval

📝 Given the following list of movies:

* **Movie 1**: captain america civil war
* **Movie 2**: captain america first avenger
* **Movie 3**: avenger infinity war
* **Movie 4**: black panther
* **Movie 5**: captain marvel
* **Movie 6**: black widow

Which movie ids are returned for the following three queries below?

Note your answer as a list of ids, e.g.: `answer = [5, 6]`

### 1.3.1 - Query I
`america OR avenger`

In [3]:
answer = [
    ### BEGIN SOLUTION
    1, 2, 3
    ### END SOLUTION
]

In [4]:
assert all([isinstance(i, int) for i in answer]), "Movie ids must be integers"
### BEGIN HIDDEN TESTS
assert sorted(answer) == [1, 2, 3]
### END HIDDEN TESTS

### 1.3.2 Query II
`captain AND NOT avenger`

In [5]:
answer = [
    ### BEGIN SOLUTION
    1, 5
    ### END SOLUTION
]

In [6]:
assert all([isinstance(i, int) for i in answer]), "Movie ids must be integers"
### BEGIN HIDDEN TESTS
assert sorted(answer) == [1, 5]
### END HIDDEN TESTS

### 1.3.3 Query III
`(marvel OR america) AND NOT (captain AND war)`

In [7]:
answer = [
    ### BEGIN SOLUTION
    2, 5
    ### END SOLUTION
]

In [8]:
assert all([isinstance(i, int) for i in answer]), "Movie ids must be integers"
### BEGIN HIDDEN TESTS
assert sorted(answer) == [2, 5]
### END HIDDEN TESTS

# Part II - Building a Boolean Search Engine for Netflix

In this week's coding task, we will build an inverted index for movies and execute multiple boolean queries on them. In the end, we will add an extension to add a ranking to our boolean search engine.

Note that all tasks below have doctests at the top of each function to help you verify your implementation before submitting the notebook. The final grading will be performed with additional tests that are not in this notebook.

But first, begin by downloading the dataset for this week into a Pandas DataFrame by executing the cell below.


## Load data

In [9]:
def load_movies(
    url: str = "https://raw.githubusercontent.com/philipphager/uva-ir0/main/data/netflix.csv"
) -> pd.DataFrame:
    df = pd.read_csv(url)
    df = df.fillna("")
    df["genres"] = df["genres"].str.split("|")
    df["directors"] = df["directors"].str.split("|")
    df["actors"] = df["actors"].str.split("|")
    return df

def test(fn: Callable):
    doctest.run_docstring_examples(fn, globals(), verbose=True, name=fn.__name__)
    
df = load_movies()
df.head()

,id,title,description,type,release_year,genres,directors,actors
0,0,five came back: the reference films,this collection includes 12 world war ii-era p...,show,1945,[documentation],[],[]
1,1,the blazing sun,a rich landlord floods and destroys a village ...,movie,1954,"[crime, romance, drama]",[youssef-chahine],"[faten-hamamah, omar-sharif, zaki-rostom, fari..."
2,2,white christmas,two talented song-and-dance men team up after ...,movie,1954,"[romance, comedy]",[michael-curtiz],"[bing-crosby, danny-kaye, rosemary-clooney, ve..."
3,3,dark waters,"ragab, a poor sailor, returns home to alexandr...",movie,1956,"[action, drama, romance, thriller]",[youssef-chahine],"[omar-sharif, faten-hamamah, ahmed-ramzy, tewf..."
4,4,cairo station,"qinawi, a physically challenged peddler who ma...",movie,1958,"[drama, crime, comedy]",[youssef-chahine],"[farid-shawqi, hind-rostom, youssef-chahine, h..."


## 2.1 Inverted Index

📝  The first coding task is to build an inverted index for tokens in the **movie title and description**.
Since we will learn text preprocessing in an upcoming week, use the provided method `tokenize(text: str) -> List[str]` to split text into individual words. Lastly, ensure that each movie id is only added once for a given token in your index (we will look into counting terms for ranking in week 3).

Create your inverted index as a Python dictionary, pointing from each token to a sorted list of movie ids:

```Python
{
    "netflix": [1029, 1038, 1155, ...],
    "original": [218, 508, 1029, ...]
    ...
}
```

</br>
<div class="alert alert-warning">
💡 Tip: The "collections.defaultdict" class might be helpful to simplify your code. But its absolutely fine to not use it to resolve this task.
</div>

In [10]:
def tokenize(text: str) -> List[str]:    
    # Lowercase all text
    text = text.lower()
    # Naively remove all punctuation from text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Naively split text into tokens on whitespace
    return text.split()

In [11]:
def create_index(df: pd.DataFrame) -> Dict[str, List[int]]:
    """
    >>> create_index(df)["witcher"]
    [3829, 4303, 4305, 4416, 4727, 5168, 5207]
    
    >>> create_index(df)["ozark"]
    [2230, 5695]
    """
    index = {}

    # Example of how to iterate over a dataframe
    for i, row in df.iterrows():
        # You can access details for each movie using the row variable:
        # E.g., row.id, row.title, row.description
        pass
    
    ### BEGIN SOLUTION
    index = defaultdict(list)
    
    for i, row in df.iterrows():
        tokens = tokenize(row.title) + tokenize(row.description)
        
        for token in set(tokens):
            index[token].append(row.id)
    ### END SOLUTION

    return index

index = create_index(df)

In [12]:
test(create_index)
### BEGIN HIDDEN TESTS
for token, postings in index.items():
    assert isinstance(token, str), f"Token not str: '{token}'"
    assert all([isinstance(p, int) for p in postings]), f"Postings not ints: {postings}"
    assert postings == sorted(postings), f"Postings for '{token}' not sorted asc: {postings}"

assert len(index) == 25889
assert index["gambit"] == [4062, 4156, 4795]
assert index["bridgerton"] == [4070, 4758]
assert index["computer"] == [144, 145, 213, 416, 1066, 1451, 1666, 3010, 3642, 3986, 5501, 5526]
### END HIDDEN TESTS

Finding tests in create_index
Trying:
    create_index(df)["witcher"]
Expecting:
    [3829, 4303, 4305, 4416, 4727, 5168, 5207]
ok
Trying:
    create_index(df)["ozark"]
Expecting:
    [2230, 5695]
ok


## 2.2 Boolean AND search

📝  Next, we use our inverted index to answer boolean AND queries (also called conjunctive queries). Complete the function `search_and` to return a list of movie titles given a list of keywords connected by AND operators. The call:

`search_and(index, ["captain", "america", "avenger"]) -> List[str]`

should corresponds to the boolean query:

`captain AND america AND avenger`

Sort the resulting movie titles alphabetically.


</br>
<div class="alert alert-warning">
💡 Tip I: Use the helper dictionary "id2title" to lookup a movie's title using its id.
<br/>
💡 Tip II: Python set operations might come in handy to quickly find common elements between two lists.
</div>

In [13]:
# Create a mapping of move ids to their title
id2title = df.set_index("id").title.to_dict()


def search_and(index: Dict, tokens: List[str]) -> List[str]:
    """
    >>> search_and(index, ["stranger", "things"])
    ['beyond stranger things', 'stranger things']
    
    >>> search_and(index, ["black", "mirror"])
    ['black mirror', 'black mirror: bandersnatch', 'death to 2020']
    
    >>> search_and(index, ["black", "mirror", "bandersnatch"])
    ['black mirror: bandersnatch']
    """
    titles = []

    ### BEGIN SOLUTION
    movie_ids = map(lambda t: set(index[t]), tokens)
    movie_ids = set.intersection(*movie_ids)
    titles = sorted(map(lambda x: id2title[x], movie_ids))
    ### END SOLUTION

    return titles

In [14]:
test(search_and)
### BEGIN HIDDEN TESTS
assert search_and(index, ["stranger", "things", "beyond"]) == ["beyond stranger things"]
assert search_and(index, ["christmas", "prince", "royal"]) == ['a christmas prince', 'a christmas prince: the royal baby', 'a christmas prince: the royal wedding', 'a princess for christmas', 'christmas with a prince']
assert search_and(index, ["monty", "python", "live"]) == ['monty python live at the hollywood bowl', 'monty python: live (mostly)', 'monty python: live at aspen', 'monty python: the meaning of live']
### END HIDDEN TESTS

Finding tests in search_and
Trying:
    search_and(index, ["stranger", "things"])
Expecting:
    ['beyond stranger things', 'stranger things']
ok
Trying:
    search_and(index, ["black", "mirror"])
Expecting:
    ['black mirror', 'black mirror: bandersnatch', 'death to 2020']
ok
Trying:
    search_and(index, ["black", "mirror", "bandersnatch"])
Expecting:
    ['black mirror: bandersnatch']
ok


## 2.3 Boolean OR search

📝  Next, complete the method `search_or` to answer disjuctive / OR queries.

`search_or(index, ["captain", "america", "avenger"]) -> List[str]`

should corresponds to the boolean query:

`captain OR america OR avenger`

Sort the resulting titles alphabetically.

In [15]:
def search_or(index: Dict, tokens: List[str]):
    """
    >>> search_or(index, ["mindhunter", "se7en"])
    ['mindhunter', 'se7en']
    
    >>> search_or(index, ["burnham", "brennan"])
    ['bo burnham: inside', 'bo burnham: make happy', 'neal brennan: 3 mics']
    """
    titles = []

    ### BEGIN SOLUTION
    movie_ids = map(lambda t: set(index[t]), tokens)
    movie_ids = set.union(*movie_ids)
    titles = sorted(map(lambda x: id2title[x], movie_ids))
    ### END SOLUTION

    return titles

In [16]:
test(search_or)
### BEGIN HIDDEN TESTS
assert search_or(index, ["burnham", "chappelle"]) == ['bo burnham: inside', 'bo burnham: make happy', 'dave chappelle', 'dave chappelle: equanimity & the bird revelation', 'dave chappelle: sticks & stones', 'dave chappelle: the closer', 'dave chappelle: the kennedy center mark twain prize', "dave chappelle: what's in a name?", 'the hall: honoring the greats of stand-up']
assert search_or(index, ["gambit", "ozark"]) == ['a farewell to ozark', "creating the queen's gambit", 'ozark', 'the netflix afterparty: the best shows of the worst year', "the queen's gambit"]
assert search_or(index, ["batman", "joker"]) == ['3 idiots', 'batman: the killing joke', 'gotham', 'lego dc comics super heroes: batman be-leaguered', 'the dark knight rises']
### END HIDDEN TESTS

Finding tests in search_or
Trying:
    search_or(index, ["mindhunter", "se7en"])
Expecting:
    ['mindhunter', 'se7en']
ok
Trying:
    search_or(index, ["burnham", "brennan"])
Expecting:
    ['bo burnham: inside', 'bo burnham: make happy', 'neal brennan: 3 mics']
ok


## 2.4 Boolean AND NOT
📝  Third, extend your answer to the conjunctive query from 2.2 above to handle a list of negated terms.

`search_and_not(index, ["queens", "gambit"], ["netflix", "afterparty"]) -> List[str]`

should corresponds to the boolean query:

`queens AND gambit AND NOT ("netflix" OR "afterparty")`

Sort the resulting titles alphabetically.

<div class="alert alert-warning">
💡 Tip: Think about how you can incorporate the "search_and" and "search_or" methods from above to make this task easier.
</div>

In [17]:
def search_and_not(index: Dict, tokens: List[str], excluded_tokens: List[str] = []):
    """    
    >>> search_and_not(index, ["stranger", "things"], ["beyond"])
    ['stranger things']
    
    >>> search_and_not(index, ["queens", "gambit"], ["netflix", "afterparty"])
    ["creating the queen's gambit", "the queen's gambit"]
    
    >>> search_and_not(index, ["queens", "gambit"], ["netflix", "afterparty", "creating"])
    ["the queen's gambit"]
    
    >>> search_and_not(index, ["batman"], ["lego"])
    ['batman: the killing joke', 'the dark knight rises']
    """
    titles = []

    ### BEGIN SOLUTION
    all_titles = search_and(index, tokens)
    excluded_titles = search_or(index, excluded_tokens)
    titles = sorted(set(all_titles) - set(excluded_titles))
    ### END SOLUTION
    
    return titles

In [18]:
test(search_and_not)
### BEGIN HIDDEN TESTS
assert search_and_not(index, ["batman"], ["lego"]) == ['batman: the killing joke', 'the dark knight rises']
assert search_and_not(index, ["stranger", "things"], ["beyond"]) == ['stranger things']
assert search_and_not(index, ["ozark"], ["farewell"]) == ["ozark"]
### END HIDDEN TESTS

Finding tests in search_and_not
Trying:
    search_and_not(index, ["stranger", "things"], ["beyond"])
Expecting:
    ['stranger things']
ok
Trying:
    search_and_not(index, ["queens", "gambit"], ["netflix", "afterparty"])
Expecting:
    ["creating the queen's gambit", "the queen's gambit"]
ok
Trying:
    search_and_not(index, ["queens", "gambit"], ["netflix", "afterparty", "creating"])
Expecting:
    ["the queen's gambit"]
ok
Trying:
    search_and_not(index, ["batman"], ["lego"])
Expecting:
    ['batman: the killing joke', 'the dark knight rises']
ok


## 2.5 Ranked Boolean Search
Lastly, we create a simple ranking for our boolean search. By definition, boolean search is NOT ranked, but returns all items that match the query. However, we can introduce a simple ranking for an OR search, for example, by listing documents that match more query terms at the top.

📝 Extend your OR search from 2.3 to return a ranked list of movies. Return not only the movie titles but the number of matching query tokens for the movie. A query `["black", "mirror"]` might result in: `[("black mirror", 2), ("black panther", 1), ...]`

Rank the resulting movies from most matching keywords to least matching keywords. When two movies have the same amount of matches, rank them ascendingly by movie title (A -> Z). Since the resulting list can get very long, return only the top k results.

<div class="alert alert-warning">
💡 Tip: The "collections.Counter" class might be a useful tool to simplify your code. Its not necessary to use it.
</div>

In [19]:
def search_ranked_or(index: Dict, tokens: List[str], top_k: int):
    """
    >>> search_ranked_or(index, ["world", "planet", "david", "attenborough"], 4)
    [('david attenborough: a life on our planet', 4), ('breaking boundaries: the science of our planet', 3), ('aerials', 2), ('dark tourist', 2)]
    
    >>> search_ranked_or(index, ["teenage", "drug", "lord", "fast"], 3)
    [('shiny_flakes: the teenage drug lord', 4), ('how to sell drugs online (fast)', 3), ('contraband', 2)]
    
    >>> search_ranked_or(index, ["black", "mirror", "2020"], 3)
    [('death to 2020', 3), ('black mirror', 2), ('black mirror: bandersnatch', 2)]
    """
    titles = []

    ### BEGIN SOLUTION
    counter = Counter()
    
    for token in tokens:
        counter.update(index[token])
    
    titles = [(id2title[m], matches) for m, matches in counter.items()]
    titles = sorted(titles, key=lambda x: (-x[1], x[0]))
    ### END SOLUTION

    return titles[:top_k]

In [20]:
test(search_ranked_or)
### BEGIN HIDDEN TESTS
assert search_ranked_or(index, ["burnham", "brennan"], 1) == [('bo burnham: inside', 1)]
assert search_ranked_or(index, ["burnham", "brennan"], 3) == [('bo burnham: inside', 1), ('bo burnham: make happy', 1), ('neal brennan: 3 mics', 1)]
assert search_ranked_or(index, ["chris", "rock", "tamborine"], 3) == [('chris rock total blackout: the tamborine extended cut', 3), ('chris rock: tamborine', 3), ('dirty daddy: the bob saget tribute', 2)]
### END HIDDEN TESTS

Finding tests in search_ranked_or
Trying:
    search_ranked_or(index, ["world", "planet", "david", "attenborough"], 4)
Expecting:
    [('david attenborough: a life on our planet', 4), ('breaking boundaries: the science of our planet', 3), ('aerials', 2), ('dark tourist', 2)]
ok
Trying:
    search_ranked_or(index, ["teenage", "drug", "lord", "fast"], 3)
Expecting:
    [('shiny_flakes: the teenage drug lord', 4), ('how to sell drugs online (fast)', 3), ('contraband', 2)]
ok
Trying:
    search_ranked_or(index, ["black", "mirror", "2020"], 3)
Expecting:
    [('death to 2020', 3), ('black mirror', 2), ('black mirror: bandersnatch', 2)]
ok


# Part III Bonus - Parametric Search

Search systems often allow for more detailed filtering of results using search parameters. These can come in useful to only show hotels in our price range, clothes in our size, or last-minute presents that ship with same-day delivery. This functionality is traditionally enabled using parametric indices, which index the meta-data of our items. In the case of movies, that might include fields like genre, author, or length.

As an optional bonus task, extend the index created in task 2.1 to contain movie meta-data information. Add the movies' `genres`, `directors`, `actors`, and `release year` to the same index by creating compound keys such as:

```Python
{
    "genre:comedy": [id_1, id_2, ...],
    "director:quentin-tarentino": [movie_ids],
    "actor:meryl-streep": [movie_ids],
    "year:2020": [movie_ids]
    ...
}
```

You should be able to use your index in the and, or, and_not functions created above to answer the three questions below.

In [21]:
def create_parametric_index(df: pd.DataFrame) -> Dict[str, List[int]]:
    """
    >>> create_parametric_index(df)["actor:meryl-streep"]
    [993, 1152, 1383, 2129, 3727, 3845, 4121, 5203]
    
    >>> create_parametric_index(df)["director:quentin-tarantino"]
    [789, 1281]
    
    >>> create_parametric_index(df)["year:1975"]
    [20, 21]
    """
    index = {}

    ### BEGIN SOLUTION
    index = defaultdict(list)
    
    for i, row in df.iterrows():
        tokens = tokenize(row.title) + tokenize(row.description)
        tokens = list(set(tokens))

        for token in tokens:
            index[token].append(row.id)

        for genre in row.genres:
            index[f"genre:{genre}"].append(row.id)

        for actor in row.actors:
            index[f"actor:{actor}"].append(row.id)

        for director in row.directors:
            index[f"director:{director}"].append(row.id)

        index[f"year:{row.release_year}"].append(row.id)
    ### END SOLUTION

    return index


parametric_index = create_parametric_index(df)

In [22]:
test(create_parametric_index)

Finding tests in create_parametric_index
Trying:
    create_parametric_index(df)["actor:meryl-streep"]
Expecting:
    [993, 1152, 1383, 2129, 3727, 3845, 4121, 5203]
ok
Trying:
    create_parametric_index(df)["director:quentin-tarantino"]
Expecting:
    [789, 1281]
ok
Trying:
    create_parametric_index(df)["year:1975"]
Expecting:
    [20, 21]
ok


### 3.2.1 Query I
Which monty python movie was released in 1975?

In [23]:
answer = search_and(
    ### BEGIN SOLUTION
    parametric_index, ["monty", "python", "year:1975"]
    ### END SOLUTION
)

In [24]:
assert all([isinstance(a, str) for a in answer]), "Expecting a list of movie titles"
### BEGIN HIDDEN TESTS
assert answer == ['monty python and the holy grail']
### END HIDDEN TESTS

### 3.2.2 Query II

Which movies were directed by Steven Spielberg or star Leonardo DiCaprio as an actor?

In [25]:
answer = search_or(
    ### BEGIN SOLUTION
    parametric_index, ["director:steven-spielberg", "actor:leonardo-dicaprio"]
    ### END SOLUTION
)

In [26]:
assert all([isinstance(a, str) for a in answer]), "Expecting a list of movie titles"
### BEGIN HIDDEN TESTS
assert answer == sorted(['the quick and the dead', 'titanic', 'catch me if you can', 'the terminal', 'war of the worlds', 'the departed', 'blood diamond', 'inception', 'django unchained', "don't look up"])
### END HIDDEN TESTS

### 3.2.3 Query III
Which comedy special of the (comedian / actor) Taylor Tomlinson was released in 2022?

In [27]:
answer = search_and(
    ### BEGIN SOLUTION
    parametric_index, ["actor:taylor-tomlinson", "year:2022", "genre:comedy"]
    ### END SOLUTION
)

In [28]:
assert all([isinstance(a, str) for a in answer]), "Expecting a list of movie titles"
### BEGIN HIDDEN TESTS
assert answer == ['taylor tomlinson: look at you']
### END HIDDEN TESTS